In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pickle
import re
from collections import defaultdict
from selenium import webdriver

In [2]:
def get_article_text(article_link):
    driver.switch_to.window(articles_handle)
    driver.get(article_link)
    article_content = " ".join([line.text for line in driver.find_elements_by_class_name('ArticleContent')[2:]])
    driver.switch_to.window(index_handle)
    return (article_content)

In [3]:
driver = webdriver.Chrome("C://Users/William/chromedriver/chromedriver.exe")

driver.execute_script("window.open('');")
index_handle, articles_handle = driver.window_handles

In [4]:
base_url = 'http://www.rodong.rep.kp/en/'
target_date = pd.to_datetime('today')
articles_dict = defaultdict(list)

while target_date >= pd.to_datetime('01-01-2018'):
    url = 'http://www.rodong.rep.kp/en/index.php?strPageID=SF01_01_03&strDate={}'.format(str(target_date).split()[0])
    driver.get(url)
    for element in driver.find_elements_by_class_name('ListNewsLineContainer'):
        article_author = element.find_element_by_class_name('ListNewsLineWriter').text
        article_title = element.find_element_by_class_name('ListNewsLineTitleW').text
        article_href = element.find_element_by_tag_name('a').get_attribute('href')
        article_text = get_article_text(base_url + re.findall("index.{0,}\d", article_href)[0])
        articles_dict[target_date.strftime('%m-%d-%Y')].append({'author': article_author,
                                                                'title': article_title,
                                                                'text': article_text})
    target_date -= dt.timedelta(1)

In [5]:
with open('rodong_articles', 'wb') as file:
    pickle.dump(articles_dict, file)

In [6]:
driver.quit()